In [91]:
import numpy as np


class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data,np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))
                    
        
        
        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            x, y = f.input, f.output
            x.grad = f.backward(y.grad)

            if x.creator is not None:
                funcs.append(x.creator)


class Function:  # 输入Variable, 输出Variable
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(as_array(y))
        output.set_creator(self)
        self.input = input
        self.output = output
        return output
# __call__方法是一个特殊的 Python方法。定义了这个方法后，当 f =
# Function()时，就可以通过编写 f(...)来调用 __call__方法了。

    def forward(self, x):
        raise NotImplementedError()

# nction类的 forward方法会抛出一个异常，目的是告诉使用了Function
# forward方法的人，这个方法应该通过继承来实现。
    def backward(self, gy):
        raise NotImplementedError()

# Function类是基类，实现所有函数通用的功能, 具体函数是在继承了 Function类的类中实现的

In [92]:
x.data = np.array(2.0)
print(x.data)

2.0


In [93]:
class Square(Function):
    def forward(self, x):
        return x**2

    def backward(self, gy):
        x = self.input.data
        gx = 2*x*gy
        return gx


x = Variable(np.array(10))
f = Square()
y = f(x)
print(type(y))
print(y.data)

<class '__main__.Variable'>
100


In [94]:
class Exp(Function):
    def forward(self, x):
        return np.exp(x)

    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x)*gy
        return gx


A = Square()
B = Exp()
C = Square()
x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)
print(y.data)

1.648721270700128


In [95]:
def numerical_diff(f, x, eps=1e-4):
    x0 = Variable(x.data-eps)
    x1 = Variable(x.data+eps)
    y0 = f(x0)
    y1 = f(x1)
    return (y1.data-y0.data)/(2*eps)


f = Square()
x = Variable(np.array(2.0))
dy = numerical_diff(f, x)
print(dy)

TypeError: <class 'numpy.float64'> is not supported

In [96]:
def f(x):
    A = Square()
    B = Exp()
    C = Square()
    return C(B(A(x)))


x = Variable(np.array(0.5))
dy = numerical_diff(f, x)
print(dy)

TypeError: <class 'numpy.float64'> is not supported

In [97]:
y.grad = np.array(1.0)
b.grad = C.backward(y.grad)
a.grad = B.backward(b.grad)
x.grad = A.backward(a.grad)
print(x.grad)

3.297442541400256


In [98]:
A = Square()
B = Exp()
C = Square()
x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)
# 反向遍历计算图的节点
assert y.creator == C
assert y.creator.input == b
assert y.creator.input.creator == B
assert y.creator.input.creator.input == a
assert y.creator.input.creator.input.creator == A
assert y.creator.input.creator.input.creator.input == x

In [99]:
A = Square()
B = Exp()
C = Square()
x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)
# 反向传播
y.grad = np.array(1.0)
y.backward()
print(x.grad)

3.297442541400256


In [100]:
def square(x):
    f = Square()
    return f(x)


def exp(x):
    f = Exp()
    return f(x)


x = Variable(np.array(0.5))
y = square(exp(square(x)))  # 连续调用
y.grad = np.array(1.0)
y.backward()
print(x.grad)

3.297442541400256


In [101]:
x = Variable(np.array(0.5))
y = square(exp(square(x)))
y.backward()
print(x.grad)

3.297442541400256


In [102]:
x=Variable(np.array(1.0))

In [103]:
x=Variable(1.0)

TypeError: <class 'float'> is not supported

In [90]:
def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x